## Handle NaNs in the brand field

Extracted the first two words from the product title and updated the missing values.

Note: In the group_similar_products.ipynb, the new and updated brand field has been cleaned up by replacing punctuation with space and converting into lower case

In [1]:
import pandas as pd
import gzip

# Reading the review
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [2]:
# Reading the image features
import struct

def readImageFeatures(path):
    f = open(path, 'rb')
    while True:
        asin = f.read(10)
        if asin == '': break
        feature = []
        for i in range(4096):
            feature.append(struct.unpack('f', f.read(4)))
        yield asin, feature

In [3]:
%cd /Users/Deepthi/Documents/DSE/Capstone/Data/From Julius

/Users/Deepthi/Documents/DSE/Capstone/Data/From Julius


In [4]:
!ls

image_features_Clothing_Shoes_and_Jewelry.b
meta_Clothing_Shoes_and_Jewelry.json.gz
ratings_Clothing_Shoes_and_Jewelry.csv
review_Women.csv
review_Women.csv.gz
review_women_brand_updated.csv
review_women_brnd_fv_prce_updated.csv
review_women_brnd_fv_updated.csv
reviews_Clothing_Shoes_and_Jewelry.json.gz
reviews_Clothing_Shoes_and_Jewelry_5.json.gz
tfidf_score.csv


In [5]:
df_meta = getDF('meta_Clothing_Shoes_and_Jewelry.json.gz')

In [6]:
df_review = getDF('reviews_Clothing_Shoes_and_Jewelry_5.json.gz')

In [7]:
df_meta_brand = df_meta[['asin', 'title','brand']].drop_duplicates()

In [8]:
## Get the first two words from the title field

df_meta_brand['brand_0'] = df_meta_brand.title.str.split().str.get(0)
df_meta_brand['brand_1'] = df_meta_brand.title.str.split().str.get(1)

In [9]:
## Combine the two words

df_meta_brand['brand_combined'] = df_meta_brand['brand_0']+' '+df_meta_brand['brand_1']

In [10]:
df_meta_brand['brand_new']=df_meta_brand.brand.fillna(df_meta_brand.brand_combined)

In [11]:
df_meta_brand[['brand','brand_new']].head(25)

,brand,brand_new
0,Big Dreams,Big Dreams
1,Boutique Cutie,Boutique Cutie
2,NaN,SHINING IMAGE
3,NaN,RiZ Women's
4,NaN,Lantin White
5,NaN,NVC Unisex
6,NaN,Kismeth Eyewear
7,NaN,Max-MPH Black
8,NaN,FX1 Small
9,NaN,Riz Small


In [12]:
## Get the file for womens category that Julius generated

review_women = pd.read_csv('review_Women.csv')

In [13]:
review_women.head()

,reviewerID,asin,unixReviewTime,brand,price
0,A4KU0XJNBH674,B000051SEN,1372723200,Suunto,159.99
1,A3HY330W94JPUQ,B000051SEN,1354492800,Suunto,159.99
2,A1CHOKV10NEI8X,B000051SEN,1143936000,Suunto,159.99
3,AOUBIY0S651IU,B000051SEN,1399248000,Suunto,159.99
4,A1GPGBHBI6T2HJ,B000051SEN,1175558400,Suunto,159.99


In [14]:
df_meta_brand.head()

,asin,title,brand,brand_0,brand_1,brand_combined,brand_new
0,0000037214,Purple Sequin Tiny Dancer Tutu Ballet Dance Fa...,Big Dreams,Purple,Sequin,Purple Sequin,Big Dreams
1,0000031887,Ballet Dress-Up Fairy Tutu,Boutique Cutie,Ballet,Dress-Up,Ballet Dress-Up,Boutique Cutie
2,0123456479,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,NaN,SHINING,IMAGE,SHINING IMAGE,SHINING IMAGE
3,0456844570,RiZ Women's Beautify Crafted &frac12; Rimmed F...,NaN,RiZ,Women's,RiZ Women's,RiZ Women's
4,0456808574,Lantin White Visor Wrap Around Ski Style Aviat...,NaN,Lantin,White,Lantin White,Lantin White


In [15]:
## Add the brand_new field into the review_women file and keep only the brand_new field

review_women_brand_updated = review_women.merge(df_meta_brand, on = 'asin', how='left')[['reviewerID','asin','unixReviewTime','brand_new','price']]

In [16]:
## Rename the brand_new field as brand

review_women_brand_updated = review_women_brand_updated.rename(columns={'brand_new':'brand'})

In [17]:
## Export the file

review_women_brand_updated.to_csv('review_women_brand_updated.csv',index=False)